In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
import uuid
import cvxpy as cp
import torch.optim as optim
import torch.nn as nn

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Step 1: Define System and Simulation Parameters
N = 64  # Number of BS antennas
K = 4   # Number of users
M = 4   # Number of RF chains
omega = 0.3  # Tradeoff weight
I_max = 10  # Maximum outer iterations
J = 10  # Can be 1, 10, or 20

SNR_dB = 12  # SNR in dB
sigma_n2 = 1  # Noise variance
P_BS = sigma_n2 * 10**(SNR_dB / 10)  # Transmit power
mu = 0.01  # Step size for analog precoder
lambda_ = 0.01  # Step size for digital precoder
L = 20  # Number of paths for channel
num_realizations = 2  # Number of channel realizations


# Dataset parameters
num_channels = 10
num_epochs = 10 if J == 1 else 3
snr_min, snr_max = 0, 12  # dB

In [3]:
# Step 2: Define Sensing Parameters
P = 3  # Number of desired sensing angles
theta_d = np.array([-60, 0, 60]) * np.pi / 180  # Desired angles in radians
delta_theta = 5 * np.pi / 180  # Half beamwidth
theta_grid = np.linspace(-np.pi / 2, np.pi / 2, 181)  # Angular grid [-90, 90] degrees
B_d = np.zeros(len(theta_grid))  # Desired beampattern
for t, theta_t in enumerate(theta_grid):
    for theta_p in theta_d:
        if abs(theta_t - theta_p) <= delta_theta:
            B_d[t] = 1

# Wavenumber and antenna spacing
lambda_wave = 1  # Wavelength (normalized)
k = 2 * np.pi / lambda_wave
d = lambda_wave / 2  # Antenna spacing

In [4]:
import torch

# Step 3: Channel Matrix Generation (Saleh-Valenzuela Model)
def generate_channel(N, M, L, device='cpu'):
    H = torch.zeros((M, N), dtype=torch.cfloat, device=device)
    for _ in range(L):
        alpha = torch.randn(2, device=device).view(torch.cfloat)[0] / torch.sqrt(torch.tensor(2.0, device=device))
        phi_r = torch.rand(1, device=device) * 2 * torch.pi
        phi_t = torch.rand(1, device=device) * 2 * torch.pi
        a_r = torch.exp(1j * k * d * torch.arange(M, dtype=torch.float32, device=device) * torch.sin(phi_r)) / torch.sqrt(torch.tensor(M, dtype=torch.float32, device=device))
        a_t = torch.exp(1j * k * d * torch.arange(N, dtype=torch.float32, device=device) * torch.sin(phi_t)) / torch.sqrt(torch.tensor(N, dtype=torch.float32, device=device))
        H += torch.sqrt(torch.tensor(N * M / L, dtype=torch.float32, device=device)) * alpha * torch.outer(a_r, a_t.conj())
    return H

# Steering vector function
def steering_vector(theta, N):
    return np.exp(1j * k * d * np.arange(N) * np.sin(theta)) / np.sqrt(N)

# Compute benchmark covariance matrix Psi
def compute_psi(N, Bd, theta_grid, PBS):
    Abar_grid = np.exp(1j * np.pi * np.arange(N)[:, None] @ np.sin(theta_grid)[None, :])
    T = 181;  
    try:
        import matlab.engine
        eng = matlab.engine.start_matlab()

        N_matlab = float(N)
        T_matlab = float(T)
        PBS_matlab = float(PBS)
        Bd_matlab = matlab.double(Bd.tolist())
        Abar_grid_matlab = matlab.double(Abar_grid.tolist(), is_complex=True)

        # 3. Execute the MATLAB CVX function
        print("Calling MATLAB CVX optimization...")
    
        # Call the function and get the result
        Psi_matlab_result = eng.solve_psi_cvx(
            N_matlab, 
            T_matlab, 
            Bd_matlab, 
            Abar_grid_matlab, 
            PBS_matlab, 
            nargout=1 # Specify that the function returns 1 output
        )
    
        # 4. Convert Result Back to NumPy
        # .array() is often the most reliable way to convert MATLAB data back to NumPy
        Psi_optimized_python = np.array(Psi_matlab_result, dtype=complex)
    
        print("Optimization successful. Resulting Psi matrix shape:", Psi_optimized_python.shape)

    except matlab.engine.EngineError as e:
        print(f"FATAL ERROR: Could not start or communicate with MATLAB Engine: {e}")
        # Handle the error by using the fallback here if needed
        Psi_optimized_python = (PBS / N) * np.eye(N, dtype=complex)

    finally:
        # 5. Stop the MATLAB Engine (CRITICAL for resource cleanup)
        if 'eng' in locals() and eng:
            eng.quit()


# Compute communication rate R
def compute_rate(H, A, D, sigma_n2):
    H_A = H @ A  # Effective channel
    R = 0
    for k in range(K):
        h_k = H_A[:, k]
        
        signal = torch.abs(h_k.conj().T @ D[:, k])**2
        interference = sum(torch.abs(h_k.conj().T @ D[:, j])**2 for j in range(K) if j != k)
        SINR = signal / (interference + sigma_n2)
        R += torch.log2(1 + SINR)
    return R

# Compute sensing error tau
def compute_tau(A, D, Psi, theta_grid):
    V = A @ D
    tau = 0
    for theta in theta_grid:
        a_theta = steering_vector(theta, N)
        # convert a_theta to torch tensor
        a_theta = torch.tensor(a_theta, dtype=torch.cfloat)
        tau += torch.abs(a_theta.conj().T @ V @ V.conj().T @ a_theta - a_theta.conj().T @ Psi @ a_theta)**2
    return tau / len(theta_grid)

def gradient_R_A(H, A, D, sigma_n2):
    xi = 1 / torch.log(torch.tensor(2.0, dtype=A.dtype, device=A.device))
    grad_A = torch.zeros_like(A, dtype=torch.cfloat)

    # Effective covariance of the digital precoder
    V = D @ D.conj().transpose(-2, -1)

    for k in range(K):
        # User-k effective channel outer product
        h_k = H[k, :].reshape(-1, 1)  # (M x 1)
        H_tilde_k = h_k @ h_k.conj().transpose(-2, -1)

        # D_bar_k = D with user k's column set to zero
        D_bar_k = D.clone()
        D_bar_k[:, k] = 0.0
        V_bar_k = D_bar_k @ D_bar_k.conj().transpose(-2, -1)

        # Denominator terms (trace parts)
        denom1 = torch.trace(A @ V @ A.conj().transpose(-2, -1) @ H_tilde_k) + sigma_n2
        denom2 = torch.trace(A @ V_bar_k @ A.conj().transpose(-2, -1) @ H_tilde_k) + sigma_n2

        # Gradient contribution
        term1 = H_tilde_k @ A @ V / denom1
        term2 = H_tilde_k @ A @ V_bar_k / denom2

        grad_A += xi * (term1 - term2)

    return grad_A

def gradient_R_D(H, A, D, sigma_n2):
    xi = 1 / torch.log(torch.tensor(2.0, dtype=A.dtype, device=A.device))
    grad_D = torch.zeros_like(D, dtype=torch.cfloat)

    for k in range(K):
        # Channel vector for user k
        h_k = H[k, :].reshape(-1, 1)  # (N x 1)
        H_tilde_k = h_k @ h_k.conj().transpose(-2, -1)

        # Effective digital-domain channel including analog precoder
        H_bar_k = A.conj().transpose(-2, -1) @ H_tilde_k @ A

        # D_bar_k = D with k-th column set to zero
        D_bar_k = D.clone()
        D_bar_k[:, k] = 0.0

        # Compute denominator terms (trace parts)
        denom1 = torch.trace(D @ D.conj().transpose(-2, -1) @ H_bar_k) + sigma_n2
        denom2 = torch.trace(D_bar_k @ D_bar_k.conj().transpose(-2, -1) @ H_bar_k) + sigma_n2

        # Compute gradient contributions
        term1 = (H_bar_k @ D) / denom1
        term2 = (H_bar_k @ D_bar_k) / denom2

        grad_D += xi * (term1 - term2)

    return grad_D

def gradient_tau_A(A, D, Psi):
    U = A @ D @ D.conj().transpose(-2, -1) @ A.conj().transpose(-2, -1)  # A D D^H A^H
    grad_A = 2 * (U - Psi) @ A @ D @ D.conj().transpose(-2, -1)
    return grad_A

def gradient_tau_D(A, D, Psi):
    U = A @ D @ D.conj().transpose(-2, -1) @ A.conj().transpose(-2, -1)  # A D D^H A^H
    grad_D = 2 * A.conj().transpose(-2, -1) @ (U - Psi) @ A @ D
    return grad_D



In [5]:

import torch

def proposed_initialization(H, theta_d, N, M, K, P_BS):

    theta_d = torch.tensor(theta_d, dtype=torch.float32, device=H.device) if isinstance(theta_d, np.ndarray) else theta_d
    steering_des = torch.exp(1j * torch.pi * torch.arange(N, dtype=torch.float32, device=H.device).reshape(-1, 1) * torch.sin(theta_d[:M-K].reshape(1, -1)))
    G = torch.cat((H.T, steering_des), dim=1)  # shape (N, K + (M-K)) = (64, 4) if M=K
    A0 = torch.exp(-1j * torch.angle(G[:, :M]))  # shape (N, M) = (64, 4)
    
    X_ZF = torch.linalg.pinv(H)  # shape (N, K) = (64, 4)
    D0 = torch.linalg.pinv(A0) @ X_ZF  # shape (M, K) = (4, 4)
    norm_factor = torch.norm(A0 @ D0, p='fro')
    D0 = torch.sqrt(torch.tensor(P_BS, dtype=A0.dtype, device=A0.device)) * D0 / norm_factor

    
    return A0, D0

def random_initialization(N, M, H, P_BS):
    A0 = np.exp(1j * np.random.uniform(0, 2 * np.pi, (N, M)))
    D0 = np.linalg.pinv(H @ A0)
    D0 = np.sqrt(P_BS) * D0 / np.linalg.norm(A0 @ D0, 'fro')
    return A0, D0

def svd_initialization(H, N, M, K, P_BS):
    _, _, Vh = svd(H, full_matrices=False)  # Vh is N x M (conjugate of right singular vectors)
    A0 = Vh.T[:, :M]  # Take first M columns, shape N x M
    A0 = np.exp(1j * np.angle(A0))  # Project to unit modulus
    H_A = H @ A0  # Shape: M x M
    try:
        D0 = np.linalg.pinv(H_A)  # Pseudoinverse of H @ A0
    except np.linalg.LinAlgError:
        D0 = np.linalg.pinv(H_A + 1e-6 * np.eye(M))  # Regularization for stability
    D0 = np.sqrt(P_BS) * D0 / np.linalg.norm(A0 @ D0, 'fro')  # Normalize
    return A0, D0

In [6]:
def project_unit_modulus(A):
    return torch.exp(1j * torch.angle(A))

def project_power_constraint(A, D, P_BS):
    norm_factor = torch.norm(A @ D, p='fro')
    D = D * (torch.sqrt(torch.tensor(P_BS, dtype=D.dtype, device=D.device)) / norm_factor)
    return D

In [7]:
def run_pga(H, A0, D0, J, I_max, mu, lambda_, omega, sigma_n2, Psi, theta_grid):
    N, K = H.shape
    A = A0.copy()
    D = D0.copy()
    objectives = []
    eta = 1 / N  # Balancing term for gradient magnitudes

    for i in range(I_max):
        print(f"\n===== Outer Iteration {i+1}/{I_max} =====")

        # ---- Inner Loop: Analog Precoder Update ----
        A_hat = A.copy()
        for j in range(J):
            grad_R_A = gradient_R_A(H, A_hat, D, sigma_n2)
            grad_tau_A = gradient_tau_A(A_hat, D, Psi)

            # Eq. (14b): Gradient Ascent on A
            grad_A = grad_R_A - omega * grad_tau_A
            A_hat = A_hat + mu * grad_A

            # Eq. (7): Unit Modulus Projection
            A_hat = project_unit_modulus(A_hat)

        A = A_hat.copy()  # Set final A after J inner updates

        # ---- Outer Loop: Digital Precoder Update ----
        grad_R_D = gradient_R_D(H, A, D, sigma_n2)
        grad_tau_D = gradient_tau_D(A, D, Psi)

        # Eq. (15): Gradient Ascent on D
        grad_D = grad_R_D - omega * eta * grad_tau_D
        D = D + lambda_ * grad_D

        # Eq. (9): Power Constraint Projection
        D = project_power_constraint(A,D, P_BS)

        # ---- Compute Objective (Eq. 5a) ----
        R = compute_rate(H, A, D, sigma_n2)
        tau = compute_tau(A, D, Psi, theta_grid)
        objective = R - omega * tau
        objectives.append(objective)

        print(f"Iteration {i+1}: R = {R:.4f}, τ = {tau:.4e}, Objective = {objective:.4f}")

    return objectives


In [8]:
import torch
import torch.nn as nn

class UPGANetLayer(nn.Module):
    def __init__(self, N, M, K, omega, J=10, eta=None):
        super(UPGANetLayer, self).__init__()
        self.J = J
        self.N, self.M, self.K = N, M, K
        self.omega = omega
        self.eta = eta if eta is not None else 1/N

        # Learnable step sizes (initialized to μ(0,0)=λ(0)=0.01)
        self.mu = nn.Parameter(torch.full((J,), 0.01, dtype=torch.float32))
        self.lambda_ = nn.Parameter(torch.tensor(0.01, dtype=torch.float32))

    def forward(self, H, A, D, Psi, sigma_n2, P_BS):
        # --- J inner updates for analog precoder ---
        for j in range(self.J):
            grad_RA = gradient_R_A(H, A, D, sigma_n2)
            grad_tauA = gradient_tau_A(A, D, Psi)
            A = A + self.mu[j] * (grad_RA - self.omega * grad_tauA)
            A = project_unit_modulus(A)

        # --- Digital precoder update ---
        grad_RD = gradient_R_D(H, A, D, sigma_n2)
        grad_tauD = gradient_tau_D(A, D, Psi)
        D = D + self.lambda_ * (grad_RD - self.omega * self.eta * grad_tauD)
        D = project_power_constraint(A, D, P_BS)

        return A, D

In [9]:
class UPGANet(nn.Module):
    def __init__(self, N, M, K, omega, I_max=120, J=10):
        super(UPGANet, self).__init__()
        self.layers = nn.ModuleList([
            UPGANetLayer(N, M, K, omega, J=J) for _ in range(I_max)
        ])
        self.I_max = I_max
        self.omega = omega

    def forward(self, H, A0, D0, Psi, sigma_n2, P_BS):
        A, D = A0, D0
        for i in range(self.I_max):
            A, D = self.layers[i](H, A, D, Psi, sigma_n2, P_BS)
        return A, D

In [10]:
def upganet_loss(H, A, D, Psi, sigma_n2, omega, theta_grid):
    R = compute_rate(H, A, D, sigma_n2)
    tau = compute_tau(A, D, Psi, theta_grid)
    return -(R - omega * tau)

In [11]:
# Instantiate model
model = UPGANet(N, M, K, omega, I_max=I_max, J=J)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    total_loss = 0.0
    for _ in range(num_channels):
        # 1. Generate random channel and Psi
        H = generate_channel(N, M, L=3)
        snr_db = np.random.uniform(snr_min, snr_max)
        P_BS = 10 ** (snr_db / 10)
        
        # FIX: unpack Psi and alpha_opt
        Psi, alpha_opt = compute_psi(N, B_d, theta_grid, P_BS)

        A0, D0 = proposed_initialization(H, theta_d, N, M, K, P_BS)

        # Ensure Psi is a proper NumPy array
        Psi = np.array(Psi, dtype=np.complex64)
        H = np.array(H, dtype=np.complex64)
        A0 = np.array(A0, dtype=np.complex64)
        D0 = np.array(D0, dtype=np.complex64)

        # Convert to tensors
        H_t = torch.tensor(H, dtype=torch.cfloat)
        Psi_t = torch.tensor(Psi, dtype=torch.cfloat)
        A0_t = torch.tensor(A0, dtype=torch.cfloat)
        D0_t = torch.tensor(D0, dtype=torch.cfloat)

        # Forward pass
        A_final, D_final = model(H_t, A0_t, D0_t, Psi_t, sigma_n2, P_BS)

        # Compute loss (negative objective)
        loss = upganet_loss(H_t, A_final, D_final, Psi_t, sigma_n2, omega, theta_grid)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoch {epoch+1}/{num_epochs}] Loss: {total_loss/num_channels:.6f}")


UnboundLocalError: cannot access local variable 'matlab' where it is not associated with a value